In [1]:
import numpy as np
import pandas as pd

import yfinance as yf

In [2]:
snp = yf.Ticker("^GSPC")

snp_hist = snp.history(start="2016-01-04", end="2022-05-26")

snp_hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04 00:00:00-05:00,2038.199951,2038.199951,1989.680054,2012.660034,4304880000,0.0,0.0
2016-01-05 00:00:00-05:00,2013.780029,2021.939941,2004.170044,2016.709961,3706620000,0.0,0.0
2016-01-06 00:00:00-05:00,2011.709961,2011.709961,1979.050049,1990.260010,4336660000,0.0,0.0
2016-01-07 00:00:00-05:00,1985.319946,1985.319946,1938.829956,1943.089966,5076590000,0.0,0.0
2016-01-08 00:00:00-05:00,1945.969971,1960.400024,1918.459961,1922.030029,4664940000,0.0,0.0
...,...,...,...,...,...,...,...
2022-05-19 00:00:00-04:00,3899.000000,3945.959961,3876.580078,3900.790039,5113550000,0.0,0.0
2022-05-20 00:00:00-04:00,3927.760010,3943.419922,3810.320068,3901.360107,5130730000,0.0,0.0
2022-05-23 00:00:00-04:00,3919.419922,3981.879883,3909.040039,3973.750000,4420030000,0.0,0.0


In [5]:
return_df = snp_hist['Open'].pct_change()
return_df

Date
2016-01-04 00:00:00-05:00         NaN
2016-01-05 00:00:00-05:00   -0.011981
2016-01-06 00:00:00-05:00   -0.001028
2016-01-07 00:00:00-05:00   -0.013118
2016-01-08 00:00:00-05:00   -0.019820
                               ...   
2022-05-19 00:00:00-04:00   -0.037754
2022-05-20 00:00:00-04:00    0.007376
2022-05-23 00:00:00-04:00   -0.002123
2022-05-24 00:00:00-04:00    0.006001
2022-05-25 00:00:00-04:00   -0.003386
Name: Open, Length: 1611, dtype: float64

In [13]:
ex = return_df.mean() - rf_rate

np.mean(ex) / np.std(ex)

/var/folders/8z/qdskhbhn44jfd2ckl12bx6xr0000gn/T/ipykernel_21057/147809612.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  np.mean(ex) / np.std(ex)


-inf

In [27]:
rf_rate = 0.003

# Calculate excess returns
excess_returns = return_df - rf_rate

# Calculate Sharpe Ratio
sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)

print("Sharpe Ratio:", sharpe_ratio)


Sharpe Ratio: -0.26078603302954395


In [36]:
def sharpe_ratio(actuals, predictions):
    returns = predictions.pct_change()
    rf_rate = 0.03
    excess_returns = returns - rf_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio

In [38]:
import numpy as np
import pandas as pd


def evaluate(prediction, ground_truth, mask, report=False):
    assert ground_truth.shape == prediction.shape, 'shape mis-match'
    performance = {}
    # mse
    performance['mse'] = np.linalg.norm((prediction - ground_truth) * mask) ** 2 / np.sum(mask)
    # IC
    df_pred = pd.DataFrame(prediction * mask)
    df_gt = pd.DataFrame(ground_truth * mask)

    ic = []
    mrr_top = 0.0
    all_miss_days_top = 0
    bt_long = 1.0
    bt_long5 = 1.0
    bt_long10 = 1.0
    irr = 0.0
    sharpe_li5 = []
    prec_10 = []

    for i in range(prediction.shape[1]):
        # IC
        ic.append(df_pred[i].corr(df_gt[i]))

        rank_gt = np.argsort(ground_truth[:, i])
        gt_top1 = set()
        gt_top5 = set()
        gt_top10 = set()

        for j in range(1, prediction.shape[0] + 1):
            cur_rank = rank_gt[-1 * j]
            if mask[cur_rank][i] < 0.5:
                continue
            if len(gt_top1) < 1:
                gt_top1.add(cur_rank)
            if len(gt_top5) < 5:
                gt_top5.add(cur_rank)
            if len(gt_top10) < 10:
                gt_top10.add(cur_rank)

        rank_pre = np.argsort(prediction[:, i])
        pre_top1 = set()
        pre_top5 = set()
        pre_top10 = set()
        for j in range(1, prediction.shape[0] + 1):
            cur_rank = rank_pre[-1 * j]
            if mask[cur_rank][i] < 0.5:
                continue
            if len(pre_top1) < 1:
                pre_top1.add(cur_rank)
            if len(pre_top5) < 5:
                pre_top5.add(cur_rank)
            if len(pre_top10) < 10:
                pre_top10.add(cur_rank)

        top1_pos_in_gt = 0
        for j in range(1, prediction.shape[0] + 1):
            cur_rank = rank_gt[-1 * j]
            if mask[cur_rank][i] < 0.5:
                continue
            else:
                top1_pos_in_gt += 1
                if cur_rank in pre_top1:
                    break
        if top1_pos_in_gt == 0:
            all_miss_days_top += 1
        else:
            mrr_top += 1.0 / top1_pos_in_gt

        real_ret_rat_top = ground_truth[list(pre_top1)[0]][i]
        bt_long += real_ret_rat_top
        gt_irr = 0.0

        for gt in gt_top10:
            gt_irr += ground_truth[gt][i]

        real_ret_rat_top5 = 0
        for pre in pre_top5:
            real_ret_rat_top5 += ground_truth[pre][i]
        irr += real_ret_rat_top5
        real_ret_rat_top5 /= 5
        bt_long5 += real_ret_rat_top5

        prec = 0.0
        real_ret_rat_top10 = 0
        for pre in pre_top10:
            real_ret_rat_top10 += ground_truth[pre][i]
            prec += (ground_truth[pre][i] >= 0)
        prec_10.append(prec / 10)
        real_ret_rat_top10 /= 10
        bt_long10 += real_ret_rat_top10
        sharpe_li5.append(real_ret_rat_top5)

    performance['IC'] = np.mean(ic)
    performance['RIC'] = np.mean(ic) / np.std(ic)
    sharpe_li5 = np.array(sharpe_li5)
    performance['sharpe5'] = (np.mean(sharpe_li5)/np.std(sharpe_li5))*15.87
    performance['prec_10'] = np.mean(prec_10)
    return performance

In [44]:
evaluate(snp_hist[['Open']], snp_hist[['Close']], snp_hist[['Volume']], report=True)

/Users/wagyu/mambaforge/envs/pattern/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


KeyError: 0